In [141]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore")

In [91]:
ruta_proyecto = 'C:/Users/mcana/OneDrive/Escritorio/PORTAFOLIO/ML/LEAD_SCORINGLEAD_SCORING_PROYECTO'

In [92]:
nombre_fichero_datos = 'Leads.csv'

In [93]:
#cargar los datos 
ruta_completa = ruta_proyecto + '/Datos/Originales/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa, index_col='id', sep=';')

In [94]:
df

,origen,fuente,no_enviar_email,no_llamar,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,ult_actividad,ambito,ocupacion,conociste_google,conociste_revista,conociste_periodico,conociste_youtube,conociste_facebook,conociste_referencias,score_actividad,score_perfil,descarga_lm
id,,,,,,,,,,,,,,,,,,,,
660737,API,Chat,No,No,0,0.0,0,0.00,Page Visited on Website,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660728,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,Business Administration,Student,No,No,No,No,No,No,14.0,20.0,Yes
660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,Media and Advertising,Unemployed,No,No,No,No,No,No,13.0,17.0,No
660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,Select,Unemployed,No,No,No,No,No,No,15.0,18.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,IT Projects Management,Unemployed,No,No,No,No,No,No,15.0,17.0,No
579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,Media and Advertising,Unemployed,No,No,No,No,No,No,14.0,19.0,Yes
579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,Business Administration,Unemployed,No,No,No,No,No,No,13.0,20.0,Yes


#### Seleccionar solo las variables finales 


In [95]:
nombre_variables_finales = ruta_proyecto + '/Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['ambito_Select',
 'descarga_lm_No',
 'fuente_Google',
 'ocupacion_Unemployed',
 'ocupacion_Working Professional',
 'origen_Landing Page Submission',
 'paginas_vistas_visita_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_total_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_Page Visited on Website',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

In [96]:
#Apuntamos las variables finales
variables_finales = ['ambito',
                    'descarga_lm',
                    'fuente',
                    'ocupacion',
                    'origen',
                    'paginas_vistas_visita',
                    'score_actividad',
                    'tiempo_en_site_total',
                     'ult_actividad',
                     'visitas_total',
                  ]

# Estructura de los Datasets

#### Eliminar registros


In [97]:
#duplicados 
df.drop_duplicates(inplace=True)

In [98]:
df.shape

(7521, 20)

In [99]:
#por EDA
df = df.loc[(df.no_llamar != 'OTROS') & (df.no_enviar_email != 'YES') & (df.ult_actividad != 'Email Bounced')]

In [100]:
df.shape

(7229, 20)

In [101]:
# separacion
x = df[variables_finales]

In [102]:
x.shape

(7229, 10)

In [103]:
#para Y
target = 'compra'

In [104]:
target

'compra'

In [105]:
#crear y 
y = df[target].copy()

In [108]:
y.shape

(7229,)

# CREAR EL PIPELINE

In [109]:
#TODOS LOS PROCESOS DE CALIDAD DE DATOS 

In [111]:
df[variables_finales].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7229 entries, 660737 to 579533
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ambito                 6585 non-null   object 
 1   descarga_lm            7229 non-null   object 
 2   fuente                 7209 non-null   object 
 3   ocupacion              5409 non-null   object 
 4   origen                 7229 non-null   object 
 5   paginas_vistas_visita  7119 non-null   float64
 6   score_actividad        4068 non-null   float64
 7   tiempo_en_site_total   7229 non-null   int64  
 8   ult_actividad          7139 non-null   object 
 9   visitas_total          7119 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 621.2+ KB


In [126]:
# CREAR LA FUNCION
def calidad_datos(df):
    temp = df.astype({'visitas_total': 'Int64'})   
    
    def imputar_moda(variable):
        return(variable.fillna(variable.mode()[0]))
    
    var_imputar_moda = ['ocupacion','ambito']
    
    temp[var_imputar_moda] = temp[var_imputar_moda].apply(imputar_moda)
    
    var_imputar_valor = ['descarga_lm',
                        'fuente',
                        'origen',
                         'ult_actividad',
                        ]
    valor = 'DESCONOCIDO'
    
    temp[var_imputar_valor] = temp[var_imputar_valor].fillna(valor)
    
    var_imputar_mediana = ['paginas_vistas_visita',
                          'score_actividad',
                          'tiempo_en_site_total',
                          'visitas_total']
    def imputar_mediana(variable): 
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))
    
    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)
    
    
    def agrupar_cat_raras(variable, criterio = 0.05):
        #calcula las frecuencias 
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio 
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #las recodifica en 'OTROS'
        temp2 = np.where(variable.isin(temp), 'OTROS',variable)
        #devuelve el resultado 
        return(temp2)

    var_agrupar_cat_raras = ['ocupacion',
                            'ambito',
                            'descarga_lm',
                            'fuente',
                            'origen',
                            'ult_actividad']
    for variable in var_agrupar_cat_raras:
        temp[variable] = agrupar_cat_raras(temp[variable], criterio = 0.02)
    
        temp['visitas_total'].clip(0,50, inplace = True)
    
    return(temp)

In [127]:
calidad_datos(x)

,ambito,descarga_lm,fuente,ocupacion,origen,paginas_vistas_visita,score_actividad,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,,
660737,Select,No,Chat,Unemployed,API,0.00,15.0,0,Page Visited on Website,0
660728,Select,No,Organic Search,Unemployed,API,2.50,15.0,674,Email Opened,5
660727,Business Administration,Yes,Direct Traffic,Student,Landing Page Submission,2.00,14.0,1532,Email Opened,2
660719,Media and Advertising,No,Direct Traffic,Unemployed,Landing Page Submission,1.00,13.0,305,OTROS,1
660681,Select,No,Google,Unemployed,Landing Page Submission,1.00,15.0,1428,Converted to Lead,2
...,...,...,...,...,...,...,...,...,...,...
579564,IT Projects Management,No,Direct Traffic,Unemployed,Landing Page Submission,2.67,15.0,1845,OTROS,8
579546,Media and Advertising,Yes,Direct Traffic,Unemployed,Landing Page Submission,2.00,14.0,238,SMS Sent,2
579545,Business Administration,Yes,Direct Traffic,Unemployed,Landing Page Submission,2.00,13.0,199,SMS Sent,2


In [128]:
x.shape

(7229, 10)

In [131]:
calidad_datos(x).isna().sum()

ambito                   0
descarga_lm              0
fuente                   0
ocupacion                0
origen                   0
paginas_vistas_visita    0
score_actividad          0
tiempo_en_site_total     0
ult_actividad            0
visitas_total            0
dtype: int64

###### convertirla en transformer
   

In [132]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

#### instancia de las transformaciones de Sklearn

In [133]:
df[variables_finales].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7229 entries, 660737 to 579533
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ambito                 6585 non-null   object 
 1   descarga_lm            7229 non-null   object 
 2   fuente                 7209 non-null   object 
 3   ocupacion              5409 non-null   object 
 4   origen                 7229 non-null   object 
 5   paginas_vistas_visita  7119 non-null   float64
 6   score_actividad        4068 non-null   float64
 7   tiempo_en_site_total   7229 non-null   int64  
 8   ult_actividad          7139 non-null   object 
 9   visitas_total          7119 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 621.2+ KB


In [147]:
#OneHotEncoding
var_ohe = ['origen', 'fuente','ult_actividad','ambito','ocupacion','descarga_lm']
#instanciamos
ohe = OneHotEncoder(sparse_output= False, handle_unknown='ignore')

#Min-Max
var_mms= ['paginas_vistas_visita', 'score_actividad','tiempo_en_site_total','visitas_total']
#instanciamos
mms = MinMaxScaler()

#### Crear el Pipeline

###### crear el column transformer

    

In [148]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (mms, var_mms),
    remainder='drop')

###### crear el pipeline del preprocesamiento


In [149]:
pipe_prepro = make_pipeline(hacer_calidad_datos,
                            ct)

### Instanciamos el modelo

#### Instanciamos el algoritmo

In [150]:
modelo = LogisticRegression(n_jobs = -1,
            solver= 'saga',
            penalty = 'l1',
            C = 1
            ) 
        
                    

##### crear el pipe final de entrenamiento


In [151]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

###### guardar el pipe final de entrenamiento

In [159]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

##### entrenar el pipe

In [160]:
nombre_pipe_ejecucion = pipe_entrenamiento.fit(x,y)

# GUARDAR EL PIPE

##### nombre del pipe final de ejecución 

In [161]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

#### GUARDAR EL PIPE FINAL DE EJECUCION


In [162]:
ruta_pipe_ejecucion = ruta_proyecto + '/Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)